In [32]:
from transformers import AutoTokenizer, AutoModel, AutoModelWithLMHead, AutoModelForMaskedLM,AutoModelForCausalLM
from transformers import pipeline
import pandas as pd
import honest_505
import warnings
warnings.filterwarnings("ignore")

In [37]:
def fill_in_and_evaluate_honest(lang: str, tokenizer, model,mask: bool, if_print_cate = True,k = 1):
    # evaluate the HONEST score for one set of parameters
    evaluator = honest_505.HonestEvaluator(lang)
    masked_templates = evaluator.templates(path = "./templates/"+ lang +"_template.tsv")
    #BERT uses [MASK] tokens
    if mask:
        # Define nlp_fill pipeline
        nlp_fill = pipeline('fill-mask', model=model, tokenizer=tokenizer, top_k=k)

        # Fill templates (please check if the filled words contain any special character)
        female_masked_templates = {masked_template: value for (masked_template, value) in masked_templates.items() if value['mf'] == 'female'}
        male_masked_templates = {masked_template: value for (masked_template, value) in masked_templates.items() if value['mf'] == 'male'}
        
        female_filled_templates = [[fill['token_str'].strip() 
                                    for fill in nlp_fill(masked_sentence.replace('[M]', tokenizer.mask_token))] 
                                    for masked_sentence in female_masked_templates.keys()]

        male_filled_templates = [[fill['token_str'].strip() 
                                  for fill in nlp_fill(masked_sentence.replace('[M]', tokenizer.mask_token))] 
                                  for masked_sentence in male_masked_templates.keys()]
    #GPT2 generates next word directly
    else:
        # Define nlp_fill pipeline
        nlp_fill = pipeline('text-generation', model=model, tokenizer=tokenizer)

        # Fill templates (please check if the filled words contain any special character)
        female_masked_templates = {masked_template: value for (masked_template, value) in masked_templates.items() if value['mf'] == 'female'}

        male_masked_templates = {masked_template: value for (masked_template, value) in masked_templates.items() if value['mf'] == 'male'}
        # Tokenize the input into integer token IDs
        
        if lang is 'zh':
            M = "[M]。"
            n = 2
        else:
            M = " [M]."
            n = 1
        inputs_female = [tokenizer.encode(masked_sentence.replace(M, ""), return_tensors='pt')  
                                    for masked_sentence in female_masked_templates.keys()]
        inputs_male = [tokenizer.encode(masked_sentence.replace(M, ""), return_tensors='pt')  
                                    for masked_sentence in male_masked_templates.keys()]

        # generate samples to predict the next word
        female_filled_sentences  = [model.generate(inputs,
                                                  Top_k =10,
                                                    do_sample=True,
                                                   repetition_penalty=1.5,
                                                   num_return_sequences = k,
                                                  max_length = inputs.shape[1] + n, pad_token_id=tokenizer.eos_token_id) for inputs in inputs_female]
        male_filled_sentences  = [model.generate(inputs,
                                              Top_k =10,
                                                do_sample=True,
                                               repetition_penalty=1.5,
                                               num_return_sequences = k,
                                              max_length = inputs.shape[1] + n, pad_token_id=tokenizer.eos_token_id) for inputs in inputs_male]

        female_filled_templates = [[tokenizer.decode(sent[-1]).strip() for sent in sents_k ] for sents_k in female_filled_sentences]
        male_filled_templates = [[tokenizer.decode(sent[-1]).strip() for sent in sents_k ] for sents_k in male_filled_sentences]
    
    # compute
    female_honest_score = evaluator.honest(female_filled_templates)
    male_honest_score = evaluator.honest(male_filled_templates)
    # print the results
    print("Top_k:",k,"language:",lang)
    print("female honest score: ", female_honest_score)
    print("male honest score: ", male_honest_score)
    # check if print out the categories
    if if_print_cate is True:
        female_honest_cate = evaluator.compute_cate(female_filled_templates)
        male_honest_cate = evaluator.compute_cate(male_filled_templates)
        print("Percentages of hurtful words about female: ", female_honest_cate)
        print("Percentages of hurtful words about male: ", male_honest_cate)
    else:
        female_honest_cate = {}
        male_honest_cate = {}
    return female_honest_score,male_honest_score,female_honest_cate,male_honest_cate

In [38]:
# test HONEST scores and update dataframes
def test_honest(tokenizer, model, lang, k_range, if_mask,honest_score_DF, cate_DF,k_cate = 15):
    # creat a list to save scores for this model
    score_list = []
    for k_num in k_range:
        # try different k
        if_print_c = (k_num is k_cate)
        fe_score, ma_score, fe_cate, ma_cate = fill_in_and_evaluate_honest(lang, 
                                                                        tokenizer=tokenizer, model=model,
                                                                        mask = if_mask,if_print_cate = if_print_c,k = k_num)
        score_list.extend([fe_score,ma_score])
        if k_num is k_cate:
            # only save when k == 15
            cate_DF = cate_DF.append(fe_cate, ignore_index=True)
            cate_DF = cate_DF.append(ma_cate, ignore_index=True)
        print("*" * 70)
    honest_score_DF.loc[len(honest_score_DF)] = score_list
    # retrun updated dataframes
    return honest_score_DF, cate_DF

In [39]:
# save test results
def save_data(lang, honest_score_df, cate_df, path = './experiment_results/'):
    honest_score_df.to_csv(path + lang + "_scores.csv")
    cate_df.to_csv(path + lang + "_categories.csv")
    print("Finished!")

In [40]:
k_range = [1,5,15]

In [41]:
# generate columns names for scores dataframe
columns_names_scores =[str(k) + "_" +gender for k in k_range for gender in ["female","male"]]

## 1.English

In [102]:
lang = 'en'
honest_score_df = pd.DataFrame(columns = columns_names_scores)
# creat a dataframe to save percentages of hurtful words for each category
cate_df = pd.DataFrame()
models_name = []

#### BERT

In [ ]:
# import the model and tokenizer 
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModelForMaskedLM.from_pretrained('bert-base-uncased')
models_name.append("BERT")
# set parameters for this model
if_mask = True

In [104]:
honest_score_df, cate_df = test_honest(tokenizer, model, lang,
                                       k_range,if_mask,
                                       honest_score_df, cate_df)

Top_k: 1 language: en
female honest score:  0.017341040462427744
male honest score:  0.013227513227513227
**********************************************************************
Top_k: 5 language: en
female honest score:  0.03236994219653179
male honest score:  0.03015873015873016
**********************************************************************
Top_k: 15 language: en
female honest score:  0.042967244701348745
male honest score:  0.032980599647266316
Percentages of hurtful words about female:  {'ddf': 0.0, 'pa': 0.031, 'asm': 0.117, 'cds': 0.224, 'qas': 0.04, 'asf': 0.009, 'om': 0.0, 'rci': 0.0, 'dmc': 0.045, 'is': 0.004, 'or': 0.094, 'ps': 0.0, 're': 0.0, 'svp': 0.031, 'ddp': 0.0, 'pr': 0.323, 'an': 0.081}
Percentages of hurtful words about male:  {'ddf': 0.0, 'pa': 0.064, 'asm': 0.134, 'cds': 0.262, 'qas': 0.064, 'asf': 0.037, 'om': 0.021, 'rci': 0.0, 'dmc': 0.128, 'is': 0.005, 'or': 0.005, 'ps': 0.0, 're': 0.011, 'svp': 0.037, 'ddp': 0.021, 'pr': 0.091, 'an': 0.118}
************

#### RoBERTa

In [105]:

# import the model and tokenizer 
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = AutoModelForMaskedLM.from_pretrained("roberta-base")
models_name.append("RoBERTa")
# set parameters for this model
if_mask = True

In [106]:
honest_score_df, cate_df = test_honest(tokenizer, model, lang,
                                       k_range,if_mask,
                                       honest_score_df, cate_df)

Top_k: 1 language: en
female honest score:  0.057803468208092484
male honest score:  0.013227513227513227
**********************************************************************
Top_k: 5 language: en
female honest score:  0.055491329479768786
male honest score:  0.049206349206349205
**********************************************************************
Top_k: 15 language: en
female honest score:  0.051252408477842
male honest score:  0.04620811287477954
Percentages of hurtful words about female:  {'ddf': 0.0, 'pa': 0.0, 'asm': 0.071, 'cds': 0.15, 'qas': 0.019, 'asf': 0.038, 'om': 0.004, 'rci': 0.0, 'dmc': 0.038, 'is': 0.004, 'or': 0.064, 'ps': 0.0, 're': 0.023, 'svp': 0.041, 'ddp': 0.0, 'pr': 0.425, 'an': 0.124}
Percentages of hurtful words about male:  {'ddf': 0.034, 'pa': 0.008, 'asm': 0.08, 'cds': 0.359, 'qas': 0.103, 'asf': 0.031, 'om': 0.034, 'rci': 0.0, 'dmc': 0.115, 'is': 0.0, 'or': 0.008, 'ps': 0.0, 're': 0.011, 'svp': 0.034, 'ddp': 0.015, 'pr': 0.084, 'an': 0.084}
*************

#### HateBERT

In [ ]:
# import the model and tokenizer 
tokenizer = AutoTokenizer.from_pretrained("GroNLP/hateBERT")
model = AutoModelForMaskedLM.from_pretrained("GroNLP/hateBERT")
models_name.append("hateBERT")
# set parameters for this model
if_mask = True

In [108]:
honest_score_df, cate_df = test_honest(tokenizer, model, lang,
                                       k_range,if_mask,
                                       honest_score_df, cate_df)

Top_k: 1 language: en
female honest score:  0.02601156069364162
male honest score:  0.037037037037037035
**********************************************************************
Top_k: 5 language: en
female honest score:  0.06242774566473988
male honest score:  0.06084656084656084
**********************************************************************
Top_k: 15 language: en
female honest score:  0.07495183044315992
male honest score:  0.06190476190476191
Percentages of hurtful words about female:  {'ddf': 0.041, 'pa': 0.0, 'asm': 0.139, 'cds': 0.129, 'qas': 0.039, 'asf': 0.015, 'om': 0.0, 'rci': 0.0, 'dmc': 0.144, 'is': 0.062, 'or': 0.044, 'ps': 0.0, 're': 0.044, 'svp': 0.005, 'ddp': 0.062, 'pr': 0.18, 'an': 0.098}
Percentages of hurtful words about male:  {'ddf': 0.009, 'pa': 0.0, 'asm': 0.214, 'cds': 0.145, 'qas': 0.057, 'asf': 0.006, 'om': 0.0, 'rci': 0.0, 'dmc': 0.16, 'is': 0.04, 'or': 0.028, 'ps': 0.0, 're': 0.028, 'svp': 0.014, 'ddp': 0.094, 'pr': 0.088, 'an': 0.117}
***************

#### GPT-2

In [109]:
# import the model and tokenizer 
tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2')
models_name.append("GPT2")
# set parameters for this model
if_mask = False

In [110]:
honest_score_df, cate_df = test_honest(tokenizer, model, lang,
                                       k_range,if_mask,
                                       honest_score_df, cate_df)

Top_k: 1 language: en
female honest score:  0.04046242774566474
male honest score:  0.042328042328042326
**********************************************************************
Top_k: 5 language: en
female honest score:  0.03410404624277456
male honest score:  0.037037037037037035
**********************************************************************
Top_k: 15 language: en
female honest score:  0.044701348747591525
male honest score:  0.031216931216931216
Percentages of hurtful words about female:  {'ddf': 0.0, 'pa': 0.0, 'asm': 0.388, 'cds': 0.043, 'qas': 0.017, 'asf': 0.009, 'om': 0.0, 'rci': 0.0, 'dmc': 0.047, 'is': 0.009, 'or': 0.022, 'ps': 0.0, 're': 0.009, 'svp': 0.013, 'ddp': 0.026, 'pr': 0.392, 'an': 0.026}
Percentages of hurtful words about male:  {'ddf': 0.0, 'pa': 0.006, 'asm': 0.463, 'cds': 0.136, 'qas': 0.034, 'asf': 0.017, 'om': 0.006, 'rci': 0.0, 'dmc': 0.056, 'is': 0.028, 'or': 0.011, 'ps': 0.0, 're': 0.017, 'svp': 0.0, 'ddp': 0.011, 'pr': 0.198, 'an': 0.017}
***********

#### xlnet

In [ ]:
# import the model and tokenizer 
tokenizer = AutoTokenizer.from_pretrained('xlnet-base-cased')
model = AutoModelForCausalLM.from_pretrained('xlnet-base-cased')
models_name.append("xlnet")
# set parameters for this model
if_mask = False

In [112]:
honest_score_df, cate_df = test_honest(tokenizer, model, lang,
                                       k_range,if_mask,
                                       honest_score_df, cate_df)

Top_k: 1 language: en
female honest score:  0.011560693641618497
male honest score:  0.013227513227513227
**********************************************************************
Top_k: 5 language: en
female honest score:  0.009826589595375723
male honest score:  0.015343915343915344
**********************************************************************
Top_k: 15 language: en
female honest score:  0.01348747591522158
male honest score:  0.009876543209876543
Percentages of hurtful words about female:  {'ddf': 0.0, 'pa': 0.0, 'asm': 0.1, 'cds': 0.9, 'qas': 0.0, 'asf': 0.0, 'om': 0.0, 'rci': 0.0, 'dmc': 0.0, 'is': 0.0, 'or': 0.0, 'ps': 0.0, 're': 0.0, 'svp': 0.0, 'ddp': 0.0, 'pr': 0.0, 'an': 0.0}
Percentages of hurtful words about male:  {'ddf': 0.0, 'pa': 0.0, 'asm': 0.089, 'cds': 0.911, 'qas': 0.0, 'asf': 0.0, 'om': 0.0, 'rci': 0.0, 'dmc': 0.0, 'is': 0.0, 'or': 0.0, 'ps': 0.0, 're': 0.0, 'svp': 0.0, 'ddp': 0.0, 'pr': 0.0, 'an': 0.0}
********************************************************

### Summary_EN

In [115]:
models_name

['BERT', 'RoBERTa', 'hateBERT', 'GPT2', 'xlnet']

In [118]:
honest_score_df.index = models_name
honest_score_df

,1_female,1_male,5_female,5_male,15_female,15_male
BERT,0.017341,0.013228,0.032370,0.030159,0.042967,0.032981
RoBERTa,0.057803,0.013228,0.055491,0.049206,0.051252,0.046208
hateBERT,0.026012,0.037037,0.062428,0.060847,0.074952,0.061905
GPT2,0.040462,0.042328,0.034104,0.037037,0.044701,0.031217
xlnet,0.011561,0.013228,0.009827,0.015344,0.013487,0.009877


In [123]:
cate_df.index = [model_name + "_" + g for model_name in models_name for g in ["female","male"]]
cate_df

,ddf,pa,asm,cds,qas,asf,om,rci,dmc,is,or,ps,re,svp,ddp,pr,an
BERT_female,0.000,0.031,0.117,0.224,0.040,0.009,0.000,0.0,0.045,0.004,0.094,0.0,0.000,0.031,0.000,0.323,0.081
BERT_male,0.000,0.064,0.134,0.262,0.064,0.037,0.021,0.0,0.128,0.005,0.005,0.0,0.011,0.037,0.021,0.091,0.118
RoBERTa_female,0.000,0.000,0.071,0.150,0.019,0.038,0.004,0.0,0.038,0.004,0.064,0.0,0.023,0.041,0.000,0.425,0.124
RoBERTa_male,0.034,0.008,0.080,0.359,0.103,0.031,0.034,0.0,0.115,0.000,0.008,0.0,0.011,0.034,0.015,0.084,0.084
hateBERT_female,0.041,0.000,0.139,0.129,0.039,0.015,0.000,0.0,0.144,0.062,0.044,0.0,0.044,0.005,0.062,0.180,0.098
hateBERT_male,0.009,0.000,0.214,0.145,0.057,0.006,0.000,0.0,0.160,0.040,0.028,0.0,0.028,0.014,0.094,0.088,0.117
GPT2_female,0.000,0.000,0.388,0.043,0.017,0.009,0.000,0.0,0.047,0.009,0.022,0.0,0.009,0.013,0.026,0.392,0.026
GPT2_male,0.000,0.006,0.463,0.136,0.034,0.017,0.006,0.0,0.056,0.028,0.011,0.0,0.017,0.000,0.011,0.198,0.017
xlnet_female,0.000,0.000,0.100,0.900,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000
xlnet_male,0.000,0.000,0.089,0.911,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.0,0.000,0.000,0.000,0.000,0.000


In [130]:
# save test results
save_data(lang, honest_score_df, cate_df)

Finished!


# 2. Chinese

In [42]:
lang = 'zh'
honest_score_df = pd.DataFrame(columns = columns_names_scores)
# creat a dataframe to save percentages of hurtful words for each category
cate_df = pd.DataFrame()
models_name = []

#### BERT_ZH

In [43]:
# import the model and tokenizer 
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
model = AutoModelForMaskedLM.from_pretrained("bert-base-chinese")
models_name.append("BERT_zh")
# set parameters for this model
if_mask = True

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [44]:
honest_score_df, cate_df = test_honest(tokenizer, model, lang,
                                       k_range,if_mask,
                                       honest_score_df, cate_df)

Top_k: 1 language: zh
female honest score:  0.002702702702702703
male honest score:  0.013227513227513227
**********************************************************************
Top_k: 5 language: zh
female honest score:  0.05891891891891892
male honest score:  0.0582010582010582
**********************************************************************
Top_k: 15 language: zh
female honest score:  0.09333333333333334
male honest score:  0.0781305114638448
Percentages of hurtful words about female:  {'re': 0.0, 'pa': 0.0, 'qas': 0.0, 'cds': 0.259, 'ddf': 0.064, 'asf': 0.069, 'ddp': 0.019, 'an': 0.033, 'ps': 0.0, 'asm': 0.002, 'svp': 0.002, 'is': 0.0, 'om': 0.114, 'pr': 0.0, 'dmc': 0.363, 'or': 0.075, 'rci': 0.0}
Percentages of hurtful words about male:  {'re': 0.0, 'pa': 0.0, 'qas': 0.0, 'cds': 0.275, 'ddf': 0.086, 'asf': 0.074, 'ddp': 0.0, 'an': 0.034, 'ps': 0.0, 'asm': 0.009, 'svp': 0.0, 'is': 0.0, 'om': 0.056, 'pr': 0.0, 'dmc': 0.296, 'or': 0.169, 'rci': 0.0}
*****************************

#### Chinese ELECTRA

In [45]:
# import the model and tokenizer 
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-electra-180g-small-ex-generator")
model = AutoModelForMaskedLM.from_pretrained("hfl/chinese-electra-180g-small-ex-generator")
models_name.append("BERT_zh")
# set parameters for this model
if_mask = True

In [46]:
honest_score_df, cate_df = test_honest(tokenizer, model, lang,
                                       k_range,if_mask,
                                       honest_score_df, cate_df)

Top_k: 1 language: zh
female honest score:  0.02702702702702703
male honest score:  0.0291005291005291
**********************************************************************
Top_k: 5 language: zh
female honest score:  0.043243243243243246
male honest score:  0.043915343915343914
**********************************************************************
Top_k: 15 language: zh
female honest score:  0.05387387387387387
male honest score:  0.04285714285714286
Percentages of hurtful words about female:  {'re': 0.0, 'pa': 0.0, 'qas': 0.0, 'cds': 0.405, 'ddf': 0.177, 'asf': 0.0, 'ddp': 0.0, 'an': 0.074, 'ps': 0.0, 'asm': 0.017, 'svp': 0.097, 'is': 0.0, 'om': 0.0, 'pr': 0.0, 'dmc': 0.224, 'or': 0.007, 'rci': 0.0}
Percentages of hurtful words about male:  {'re': 0.0, 'pa': 0.0, 'qas': 0.0, 'cds': 0.412, 'ddf': 0.206, 'asf': 0.004, 'ddp': 0.0, 'an': 0.058, 'ps': 0.0, 'asm': 0.012, 'svp': 0.099, 'is': 0.0, 'om': 0.0, 'pr': 0.0, 'dmc': 0.156, 'or': 0.053, 'rci': 0.0}
**********************************

#### Chinese-xlnet

In [47]:
# import the model and tokenizer 
tokenizer = AutoTokenizer.from_pretrained('xlnet-base-cased')
model = AutoModelForCausalLM.from_pretrained('xlnet-base-cased')
models_name.append("xlnet_zh")
# set parameters for this model
if_mask = False

In [48]:
honest_score_df, cate_df = test_honest(tokenizer, model, lang,
                                       k_range,if_mask,
                                       honest_score_df, cate_df)

Top_k: 1 language: zh
female honest score:  0.0
male honest score:  0.0
**********************************************************************
Top_k: 5 language: zh
female honest score:  0.0
male honest score:  0.0
**********************************************************************
Top_k: 15 language: zh
female honest score:  0.0
male honest score:  0.0
Percentages of hurtful words about female:  {'re': nan, 'pa': nan, 'qas': nan, 'cds': nan, 'ddf': nan, 'asf': nan, 'ddp': nan, 'an': nan, 'ps': nan, 'asm': nan, 'svp': nan, 'is': nan, 'om': nan, 'pr': nan, 'dmc': nan, 'or': nan, 'rci': nan}
Percentages of hurtful words about male:  {'re': nan, 'pa': nan, 'qas': nan, 'cds': nan, 'ddf': nan, 'asf': nan, 'ddp': nan, 'an': nan, 'ps': nan, 'asm': nan, 'svp': nan, 'is': nan, 'om': nan, 'pr': nan, 'dmc': nan, 'or': nan, 'rci': nan}
**********************************************************************


#### traditional Chinese GPT2

In [ ]:
# import the model and tokenizer 
tokenizer = AutoTokenizer.from_pretrained('ckiplab/gpt2-base-chinese')
model = AutoModelForCausalLM.from_pretrained('ckiplab/gpt2-base-chinese')
models_name.append("gpt2_zh")
# set parameters for this model
if_mask = False

In [ ]:
honest_score_df, cate_df = test_honest(tokenizer, model, lang,
                                       k_range,if_mask,
                                       honest_score_df, cate_df)

### Summary_zh

In [52]:
models_name

['BERT_zh', 'BERT_zh', 'xlnet_zh', 'gpt2_zh']

In [53]:
honest_score_df.index = models_name
honest_score_df

,1_female,1_male,5_female,5_male,15_female,15_male
BERT_zh,0.002703,0.013228,0.058919,0.058201,0.093333,0.078131
BERT_zh,0.027027,0.029101,0.043243,0.043915,0.053874,0.042857
xlnet_zh,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
gpt2_zh,0.054054,0.055556,0.048108,0.053439,0.053514,0.056437


In [54]:
cate_df.index = [model_name + "_" + g for model_name in models_name for g in ["female","male"]]
cate_df

,re,pa,qas,cds,ddf,asf,ddp,an,ps,asm,svp,is,om,pr,dmc,or,rci
BERT_zh_female,0.0,0.0,0.000,0.259,0.064,0.069,0.019,0.033,0.000,0.002,0.002,0.0,0.114,0.0,0.363,0.075,0.0
BERT_zh_male,0.0,0.0,0.000,0.275,0.086,0.074,0.000,0.034,0.000,0.009,0.000,0.0,0.056,0.0,0.296,0.169,0.0
BERT_zh_female,0.0,0.0,0.000,0.405,0.177,0.000,0.000,0.074,0.000,0.017,0.097,0.0,0.000,0.0,0.224,0.007,0.0
BERT_zh_male,0.0,0.0,0.000,0.412,0.206,0.004,0.000,0.058,0.000,0.012,0.099,0.0,0.000,0.0,0.156,0.053,0.0
xlnet_zh_female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
xlnet_zh_male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
gpt2_zh_female,0.0,0.0,0.007,0.051,0.000,0.108,0.104,0.000,0.024,0.030,0.189,0.0,0.000,0.0,0.488,0.000,0.0
gpt2_zh_male,0.0,0.0,0.003,0.072,0.000,0.059,0.147,0.000,0.022,0.050,0.172,0.0,0.000,0.0,0.475,0.000,0.0


In [55]:
# save test results
save_data(lang, honest_score_df, cate_df)

Finished!


# 3. French

In [133]:
lang = 'fr'
honest_score_df = pd.DataFrame(columns = columns_names_scores)
# creat a dataframe to save percentages of hurtful words for each category
cate_df = pd.DataFrame()
models_name = []

#### CamemBERT

In [ ]:
# import the model and tokenizer 
tokenizer = AutoTokenizer.from_pretrained("camembert-base")
model = AutoModelForMaskedLM.from_pretrained("camembert-base")
models_name.append("CamemBERT")
# set parameters for this model
if_mask = True

In [135]:
honest_score_df, cate_df = test_honest(tokenizer, model, lang,
                                       k_range,if_mask,
                                       honest_score_df, cate_df)

Top_k: 1 language: fr
female honest score:  0.19796954314720813
male honest score:  0.1794871794871795
**********************************************************************
Top_k: 5 language: fr
female honest score:  0.10101522842639594
male honest score:  0.09384615384615384
**********************************************************************
Top_k: 15 language: fr
female honest score:  0.07021996615905245
male honest score:  0.07487179487179488
Percentages of hurtful words about female:  {'ddf': 0.002, 'pa': 0.0, 'cds': 0.214, 'asm': 0.241, 'qas': 0.002, 'asf': 0.002, 'om': 0.0, 'rci': 0.0, 'dmc': 0.019, 'is': 0.0, 'or': 0.048, 'ps': 0.0, 're': 0.022, 'svp': 0.0, 'ddp': 0.002, 'pr': 0.441, 'an': 0.005}
Percentages of hurtful words about male:  {'ddf': 0.0, 'pa': 0.005, 'cds': 0.397, 'asm': 0.176, 'qas': 0.002, 'asf': 0.009, 'om': 0.023, 'rci': 0.0, 'dmc': 0.091, 'is': 0.0, 'or': 0.078, 'ps': 0.0, 're': 0.089, 'svp': 0.043, 'ddp': 0.007, 'pr': 0.048, 'an': 0.032}
******************

#### CamemBERT trained in Wikipedia

In [ ]:
# import the model and tokenizer 
tokenizer = AutoTokenizer.from_pretrained("camembert-base")
model = AutoModelForMaskedLM.from_pretrained("camembert/camembert-base-wikipedia-4gb")
models_name.append("CamemBERT_Wiki")
# set parameters for this model
if_mask = True

In [138]:
honest_score_df, cate_df = test_honest(tokenizer, model, lang,
                                       k_range,if_mask,
                                       honest_score_df, cate_df)

Top_k: 1 language: fr
female honest score:  0.012690355329949238
male honest score:  0.01282051282051282
**********************************************************************
Top_k: 5 language: fr
female honest score:  0.04568527918781726
male honest score:  0.03435897435897436
**********************************************************************
Top_k: 15 language: fr
female honest score:  0.033164128595600674
male honest score:  0.033504273504273506
Percentages of hurtful words about female:  {'ddf': 0.0, 'pa': 0.0, 'cds': 0.128, 'asm': 0.036, 'qas': 0.041, 'asf': 0.01, 'om': 0.005, 'rci': 0.0, 'dmc': 0.051, 'is': 0.0, 'or': 0.684, 'ps': 0.0, 're': 0.005, 'svp': 0.02, 'ddp': 0.01, 'pr': 0.01, 'an': 0.0}
Percentages of hurtful words about male:  {'ddf': 0.0, 'pa': 0.0, 'cds': 0.158, 'asm': 0.061, 'qas': 0.01, 'asf': 0.01, 'om': 0.0, 'rci': 0.0, 'dmc': 0.02, 'is': 0.0, 'or': 0.673, 'ps': 0.0, 're': 0.015, 'svp': 0.051, 'ddp': 0.0, 'pr': 0.0, 'an': 0.0}
*******************************

#### BelGPT-2

In [ ]:
# import the model and tokenizer 
tokenizer = AutoTokenizer.from_pretrained("antoiloui/belgpt2")
model = AutoModelForCausalLM.from_pretrained("antoiloui/belgpt2")
models_name.append("BelGPT2")
# set parameters for this model
if_mask = False

In [142]:
honest_score_df, cate_df = test_honest(tokenizer, model, lang,
                                       k_range,if_mask,
                                       honest_score_df, cate_df)

Top_k: 1 language: fr
female honest score:  0.04568527918781726
male honest score:  0.05384615384615385
**********************************************************************
Top_k: 5 language: fr
female honest score:  0.04060913705583756
male honest score:  0.05897435897435897
**********************************************************************
Top_k: 15 language: fr
female honest score:  0.03857868020304569
male honest score:  0.05384615384615385
Percentages of hurtful words about female:  {'ddf': 0.0, 'pa': 0.0, 'cds': 0.311, 'asm': 0.285, 'qas': 0.004, 'asf': 0.013, 'om': 0.0, 'rci': 0.0, 'dmc': 0.048, 'is': 0.0, 'or': 0.018, 'ps': 0.0, 're': 0.0, 'svp': 0.0, 'ddp': 0.004, 'pr': 0.307, 'an': 0.009}
Percentages of hurtful words about male:  {'ddf': 0.0, 'pa': 0.0, 'cds': 0.317, 'asm': 0.238, 'qas': 0.003, 'asf': 0.025, 'om': 0.003, 'rci': 0.0, 'dmc': 0.092, 'is': 0.0, 'or': 0.251, 'ps': 0.0, 're': 0.006, 'svp': 0.022, 'ddp': 0.013, 'pr': 0.003, 'an': 0.025}
***********************

#### FlauBERT

In [ ]:
# import the model and tokenizer 
tokenizer = AutoTokenizer.from_pretrained("flaubert/flaubert_base_uncased")
model = AutoModelForMaskedLM.from_pretrained("flaubert/flaubert_base_uncased")
models_name.append("FlauBert")
# set parameters for this model
if_mask = True

In [144]:
honest_score_df, cate_df = test_honest(tokenizer, model, lang,
                                       k_range,if_mask,
                                       honest_score_df, cate_df)

Top_k: 1 language: fr
female honest score:  0.05583756345177665
male honest score:  0.002564102564102564
**********************************************************************
Top_k: 5 language: fr
female honest score:  0.03705583756345178
male honest score:  0.009743589743589744
**********************************************************************
Top_k: 15 language: fr
female honest score:  0.025888324873096447
male honest score:  0.013675213675213675
Percentages of hurtful words about female:  {'ddf': 0.0, 'pa': 0.0, 'cds': 0.458, 'asm': 0.209, 'qas': 0.007, 'asf': 0.026, 'om': 0.0, 'rci': 0.0, 'dmc': 0.092, 'is': 0.0, 'or': 0.013, 'ps': 0.0, 're': 0.0, 'svp': 0.007, 'ddp': 0.033, 'pr': 0.105, 'an': 0.052}
Percentages of hurtful words about male:  {'ddf': 0.0, 'pa': 0.0, 'cds': 0.375, 'asm': 0.3, 'qas': 0.012, 'asf': 0.012, 'om': 0.0, 'rci': 0.0, 'dmc': 0.125, 'is': 0.0, 'or': 0.025, 'ps': 0.0, 're': 0.0, 'svp': 0.05, 'ddp': 0.05, 'pr': 0.0, 'an': 0.05}
****************************

### Summary_FR

In [145]:
models_name

['CamemBERT', 'CamemBERT_Wiki', 'BelGPT2', 'FlauBert']

In [146]:
honest_score_df.index = models_name
honest_score_df

,1_female,1_male,5_female,5_male,15_female,15_male
CamemBERT,0.197970,0.179487,0.101015,0.093846,0.070220,0.074872
CamemBERT_Wiki,0.012690,0.012821,0.045685,0.034359,0.033164,0.033504
BelGPT2,0.045685,0.053846,0.040609,0.058974,0.038579,0.053846
FlauBert,0.055838,0.002564,0.037056,0.009744,0.025888,0.013675


In [147]:
cate_df.index = [model_name + "_" + g for model_name in models_name for g in ["female","male"]]
cate_df

,ddf,pa,cds,asm,qas,asf,om,rci,dmc,is,or,ps,re,svp,ddp,pr,an
CamemBERT_female,0.002,0.000,0.214,0.241,0.002,0.002,0.000,0.0,0.019,0.0,0.048,0.0,0.022,0.000,0.002,0.441,0.005
CamemBERT_male,0.000,0.005,0.397,0.176,0.002,0.009,0.023,0.0,0.091,0.0,0.078,0.0,0.089,0.043,0.007,0.048,0.032
CamemBERT_Wiki_female,0.000,0.000,0.128,0.036,0.041,0.010,0.005,0.0,0.051,0.0,0.684,0.0,0.005,0.020,0.010,0.010,0.000
CamemBERT_Wiki_male,0.000,0.000,0.158,0.061,0.010,0.010,0.000,0.0,0.020,0.0,0.673,0.0,0.015,0.051,0.000,0.000,0.000
BelGPT2_female,0.000,0.000,0.311,0.285,0.004,0.013,0.000,0.0,0.048,0.0,0.018,0.0,0.000,0.000,0.004,0.307,0.009
BelGPT2_male,0.000,0.000,0.317,0.238,0.003,0.025,0.003,0.0,0.092,0.0,0.251,0.0,0.006,0.022,0.013,0.003,0.025
FlauBert_female,0.000,0.000,0.458,0.209,0.007,0.026,0.000,0.0,0.092,0.0,0.013,0.0,0.000,0.007,0.033,0.105,0.052
FlauBert_male,0.000,0.000,0.375,0.300,0.012,0.012,0.000,0.0,0.125,0.0,0.025,0.0,0.000,0.050,0.050,0.000,0.050


In [148]:
# save test results
save_data(lang, honest_score_df, cate_df)

Finished!


# 4.Spanish

In [151]:
lang = 'es'
honest_score_df = pd.DataFrame(columns = columns_names_scores)
# creat a dataframe to save percentages of hurtful words for each category
cate_df = pd.DataFrame()
models_name = []

#### RoBERTa_es

In [152]:
# import the model and tokenizer 
tokenizer = AutoTokenizer.from_pretrained("PlanTL-GOB-ES/roberta-base-bne")
model = AutoModelForMaskedLM.from_pretrained("PlanTL-GOB-ES/roberta-base-bne")
models_name.append("RoBERTa_es")
# set parameters for this model
if_mask = True

In [153]:
honest_score_df, cate_df = test_honest(tokenizer, model, lang,
                                       k_range,if_mask,
                                       honest_score_df, cate_df)

Top_k: 1 language: es
female honest score:  0.007692307692307693
male honest score:  0.010256410256410256
**********************************************************************
Top_k: 5 language: es
female honest score:  0.03076923076923077
male honest score:  0.03076923076923077
**********************************************************************
Top_k: 15 language: es
female honest score:  0.042222222222222223
male honest score:  0.03230769230769231
Percentages of hurtful words about female:  {'ddf': 0.0, 'pa': 0.0, 'pr': 0.219, 'cds': 0.186, 'asf': 0.259, 'asm': 0.085, 'qas': 0.045, 'rci': 0.0, 'dmc': 0.053, 'is': 0.0, 'or': 0.0, 'ps': 0.0, 're': 0.0, 'svp': 0.0, 'ddp': 0.004, 'om': 0.036, 'an': 0.113}
Percentages of hurtful words about male:  {'ddf': 0.037, 'pa': 0.0, 'pr': 0.021, 'cds': 0.27, 'asf': 0.254, 'asm': 0.074, 'qas': 0.005, 'rci': 0.0, 'dmc': 0.016, 'is': 0.005, 'or': 0.0, 'ps': 0.0, 're': 0.0, 'svp': 0.0, 'ddp': 0.011, 'om': 0.063, 'an': 0.243}
***********************

#### gpt2-spanish

In [ ]:
# import the model and tokenizer 
tokenizer = AutoTokenizer.from_pretrained("DeepESP/gpt2-spanish")
model = AutoModelForCausalLM.from_pretrained("DeepESP/gpt2-spanish")
models_name.append("GPT2_es")
# set parameters for this model
if_mask = False

In [155]:
honest_score_df, cate_df = test_honest(tokenizer, model, lang,
                                       k_range,if_mask,
                                       honest_score_df, cate_df)

Top_k: 1 language: es
female honest score:  0.035897435897435895
male honest score:  0.017948717948717947
**********************************************************************
Top_k: 5 language: es
female honest score:  0.02923076923076923
male honest score:  0.021538461538461538
**********************************************************************
Top_k: 15 language: es
female honest score:  0.032136752136752135
male honest score:  0.023247863247863248
Percentages of hurtful words about female:  {'ddf': 0.0, 'pa': 0.0, 'pr': 0.372, 'cds': 0.069, 'asf': 0.218, 'asm': 0.021, 'qas': 0.053, 'rci': 0.0, 'dmc': 0.027, 'is': 0.011, 'or': 0.0, 'ps': 0.0, 're': 0.0, 'svp': 0.0, 'ddp': 0.021, 'om': 0.016, 'an': 0.191}
Percentages of hurtful words about male:  {'ddf': 0.015, 'pa': 0.0, 'pr': 0.088, 'cds': 0.39, 'asf': 0.074, 'asm': 0.007, 'qas': 0.029, 'rci': 0.0, 'dmc': 0.213, 'is': 0.007, 'or': 0.007, 'ps': 0.0, 're': 0.0, 'svp': 0.0, 'ddp': 0.022, 'om': 0.044, 'an': 0.103}
*****************

### Summary_ES

In [156]:
models_name

['RoBERTa_es', 'GPT2_es']

In [157]:
honest_score_df.index = models_name
honest_score_df

,1_female,1_male,5_female,5_male,15_female,15_male
RoBERTa_es,0.007692,0.010256,0.030769,0.030769,0.042222,0.032308
GPT2_es,0.035897,0.017949,0.029231,0.021538,0.032137,0.023248


In [158]:
cate_df.index = [model_name + "_" + g for model_name in models_name for g in ["female","male"]]
cate_df

,ddf,pa,pr,cds,asf,asm,qas,rci,dmc,is,or,ps,re,svp,ddp,om,an
RoBERTa_es_female,0.000,0.0,0.219,0.186,0.259,0.085,0.045,0.0,0.053,0.000,0.000,0.0,0.0,0.0,0.004,0.036,0.113
RoBERTa_es_male,0.037,0.0,0.021,0.270,0.254,0.074,0.005,0.0,0.016,0.005,0.000,0.0,0.0,0.0,0.011,0.063,0.243
GPT2_es_female,0.000,0.0,0.372,0.069,0.218,0.021,0.053,0.0,0.027,0.011,0.000,0.0,0.0,0.0,0.021,0.016,0.191
GPT2_es_male,0.015,0.0,0.088,0.390,0.074,0.007,0.029,0.0,0.213,0.007,0.007,0.0,0.0,0.0,0.022,0.044,0.103


In [159]:
# save test results
save_data(lang, honest_score_df, cate_df)

Finished!


# 5. Italian

In [160]:
lang = 'it'
honest_score_df = pd.DataFrame(columns = columns_names_scores)
# creat a dataframe to save percentages of hurtful words for each category
cate_df = pd.DataFrame()
models_name = []

#### BERT_it

In [ ]:
# import the model and tokenizer 
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-italian-uncased")
model = AutoModelForMaskedLM.from_pretrained("dbmdz/bert-base-italian-uncased")
models_name.append("BERT_it")
# set parameters for this model
if_mask = True

In [162]:
honest_score_df, cate_df = test_honest(tokenizer, model, lang,
                                       k_range,if_mask,
                                       honest_score_df, cate_df)

Top_k: 1 language: it
female honest score:  0.04938271604938271
male honest score:  0.03950617283950617
**********************************************************************
Top_k: 5 language: it
female honest score:  0.058271604938271604
male honest score:  0.027160493827160494
**********************************************************************
Top_k: 15 language: it
female honest score:  0.04691358024691358
male honest score:  0.02633744855967078
Percentages of hurtful words about female:  {'ddf': 0.0, 'pa': 0.0, 'cds': 0.2, 'asm': 0.011, 'qas': 0.0, 'asf': 0.267, 'om': 0.004, 'rci': 0.0, 'dmc': 0.0, 'is': 0.0, 'or': 0.0, 'ps': 0.0, 're': 0.0, 'svp': 0.0, 'ddp': 0.105, 'pr': 0.291, 'an': 0.123}
Percentages of hurtful words about male:  {'ddf': 0.012, 'pa': 0.0, 'cds': 0.594, 'asm': 0.05, 'qas': 0.0, 'asf': 0.0, 'om': 0.144, 'rci': 0.0, 'dmc': 0.062, 'is': 0.019, 'or': 0.0, 'ps': 0.081, 're': 0.0, 'svp': 0.0, 'ddp': 0.012, 'pr': 0.0, 'an': 0.025}
**********************************

#### GePpeTto

In [ ]:
# import the model and tokenizer 
tokenizer = AutoTokenizer.from_pretrained("LorenzoDeMattei/GePpeTto")
model = AutoModelForCausalLM.from_pretrained("LorenzoDeMattei/GePpeTto")
models_name.append("GePpeTto")
# set parameters for this model
if_mask = False

In [164]:
honest_score_df, cate_df = test_honest(tokenizer, model, lang,
                                       k_range,if_mask,
                                       honest_score_df, cate_df)

Top_k: 1 language: it
female honest score:  0.027160493827160494
male honest score:  0.022222222222222223
**********************************************************************
Top_k: 5 language: it
female honest score:  0.04049382716049383
male honest score:  0.02271604938271605
**********************************************************************
Top_k: 15 language: it
female honest score:  0.03637860082304527
male honest score:  0.023868312757201648
Percentages of hurtful words about female:  {'ddf': 0.0, 'pa': 0.0, 'cds': 0.276, 'asm': 0.0, 'qas': 0.0, 'asf': 0.339, 'om': 0.0, 'rci': 0.0, 'dmc': 0.0, 'is': 0.0, 'or': 0.0, 'ps': 0.0, 're': 0.0, 'svp': 0.258, 'ddp': 0.045, 'pr': 0.081, 'an': 0.0}
Percentages of hurtful words about male:  {'ddf': 0.028, 'pa': 0.0, 'cds': 0.317, 'asm': 0.021, 'qas': 0.007, 'asf': 0.007, 'om': 0.172, 'rci': 0.0, 'dmc': 0.021, 'is': 0.034, 'or': 0.007, 'ps': 0.076, 're': 0.007, 'svp': 0.255, 'ddp': 0.0, 'pr': 0.007, 'an': 0.041}
************************

### Summary_It

In [165]:
models_name

['BERT_it', 'GePpeTto']

In [166]:
honest_score_df.index = models_name
honest_score_df

,1_female,1_male,5_female,5_male,15_female,15_male
BERT_it,0.049383,0.039506,0.058272,0.027160,0.046914,0.026337
GePpeTto,0.027160,0.022222,0.040494,0.022716,0.036379,0.023868


In [167]:
cate_df.index = [model_name + "_" + g for model_name in models_name for g in ["female","male"]]
cate_df

,ddf,pa,cds,asm,qas,asf,om,rci,dmc,is,or,ps,re,svp,ddp,pr,an
BERT_it_female,0.000,0.0,0.200,0.011,0.000,0.267,0.004,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.105,0.291,0.123
BERT_it_male,0.012,0.0,0.594,0.050,0.000,0.000,0.144,0.0,0.062,0.019,0.000,0.081,0.000,0.000,0.012,0.000,0.025
GePpeTto_female,0.000,0.0,0.276,0.000,0.000,0.339,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.258,0.045,0.081,0.000
GePpeTto_male,0.028,0.0,0.317,0.021,0.007,0.007,0.172,0.0,0.021,0.034,0.007,0.076,0.007,0.255,0.000,0.007,0.041


In [168]:
# save test results
save_data(lang, honest_score_df, cate_df)

Finished!


# 6. Portuguese

In [169]:
lang = 'pt'
honest_score_df = pd.DataFrame(columns = columns_names_scores)
# creat a dataframe to save percentages of hurtful words for each category
cate_df = pd.DataFrame()
models_name = []

#### BERT_it

In [ ]:
# import the model and tokenizer 
tokenizer = AutoTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")
model = AutoModelForMaskedLM.from_pretrained("neuralmind/bert-base-portuguese-cased")
models_name.append("BERT_pt")
# set parameters for this model
if_mask = True

In [171]:
honest_score_df, cate_df = test_honest(tokenizer, model, lang,
                                       k_range,if_mask,
                                       honest_score_df, cate_df)

Top_k: 1 language: pt
female honest score:  0.044226044226044224
male honest score:  0.011904761904761904
**********************************************************************
Top_k: 5 language: pt
female honest score:  0.06928746928746929
male honest score:  0.023333333333333334
**********************************************************************
Top_k: 15 language: pt
female honest score:  0.05110565110565111
male honest score:  0.030634920634920636
Percentages of hurtful words about female:  {'ddf': 0.0, 'pa': 0.0, 'asf': 0.356, 'cds': 0.529, 'asm': 0.048, 'qas': 0.003, 'om': 0.003, 'rci': 0.0, 'dmc': 0.0, 'is': 0.022, 'or': 0.0, 'ps': 0.0, 're': 0.0, 'svp': 0.0, 'ddp': 0.006, 'pr': 0.032, 'an': 0.0}
Percentages of hurtful words about male:  {'ddf': 0.0, 'pa': 0.0, 'asf': 0.057, 'cds': 0.606, 'asm': 0.104, 'qas': 0.0, 'om': 0.01, 'rci': 0.0, 'dmc': 0.005, 'is': 0.052, 'or': 0.005, 'ps': 0.0, 're': 0.01, 'svp': 0.005, 'ddp': 0.088, 'pr': 0.041, 'an': 0.016}
***********************

#### GPT2_pt

In [ ]:
# import the model and tokenizer 
tokenizer = AutoTokenizer.from_pretrained("pierreguillou/gpt2-small-portuguese")
model = AutoModelForCausalLM.from_pretrained("pierreguillou/gpt2-small-portuguese")
models_name.append("GPT2_pt")
# set parameters for this model
if_mask = False

In [173]:
honest_score_df, cate_df = test_honest(tokenizer, model, lang,
                                       k_range,if_mask,
                                       honest_score_df, cate_df)

Top_k: 1 language: pt
female honest score:  0.029484029484029485
male honest score:  0.011904761904761904
**********************************************************************
Top_k: 5 language: pt
female honest score:  0.023095823095823097
male honest score:  0.006666666666666667
**********************************************************************
Top_k: 15 language: pt
female honest score:  0.025552825552825554
male honest score:  0.008888888888888889
Percentages of hurtful words about female:  {'ddf': 0.0, 'pa': 0.0, 'asf': 0.449, 'cds': 0.391, 'asm': 0.006, 'qas': 0.0, 'om': 0.0, 'rci': 0.0, 'dmc': 0.0, 'is': 0.0, 'or': 0.0, 'ps': 0.0, 're': 0.0, 'svp': 0.0, 'ddp': 0.019, 'pr': 0.096, 'an': 0.038}
Percentages of hurtful words about male:  {'ddf': 0.0, 'pa': 0.0, 'asf': 0.0, 'cds': 0.696, 'asm': 0.161, 'qas': 0.0, 'om': 0.0, 'rci': 0.0, 'dmc': 0.054, 'is': 0.0, 'or': 0.0, 'ps': 0.0, 're': 0.0, 'svp': 0.0, 'ddp': 0.054, 'pr': 0.036, 'an': 0.0}
*************************************

### Summary_PT

In [174]:
models_name

['BERT_pt', 'GPT2_pt']

In [175]:
honest_score_df.index = models_name
honest_score_df

,1_female,1_male,5_female,5_male,15_female,15_male
BERT_pt,0.044226,0.011905,0.069287,0.023333,0.051106,0.030635
GPT2_pt,0.029484,0.011905,0.023096,0.006667,0.025553,0.008889


In [176]:
cate_df.index = [model_name + "_" + g for model_name in models_name for g in ["female","male"]]
cate_df

,ddf,pa,asf,cds,asm,qas,om,rci,dmc,is,or,ps,re,svp,ddp,pr,an
BERT_pt_female,0.0,0.0,0.356,0.529,0.048,0.003,0.003,0.0,0.000,0.022,0.000,0.0,0.00,0.000,0.006,0.032,0.000
BERT_pt_male,0.0,0.0,0.057,0.606,0.104,0.000,0.010,0.0,0.005,0.052,0.005,0.0,0.01,0.005,0.088,0.041,0.016
GPT2_pt_female,0.0,0.0,0.449,0.391,0.006,0.000,0.000,0.0,0.000,0.000,0.000,0.0,0.00,0.000,0.019,0.096,0.038
GPT2_pt_male,0.0,0.0,0.000,0.696,0.161,0.000,0.000,0.0,0.054,0.000,0.000,0.0,0.00,0.000,0.054,0.036,0.000


In [177]:
# save test results
save_data(lang, honest_score_df, cate_df)

Finished!


# 7. Romanian

In [178]:
lang = 'ro'
honest_score_df = pd.DataFrame(columns = columns_names_scores)
# creat a dataframe to save percentages of hurtful words for each category
cate_df = pd.DataFrame()
models_name = []

#### BERT_Ro

In [ ]:
# import the model and tokenizer 
tokenizer = AutoTokenizer.from_pretrained("dumitrescustefan/bert-base-romanian-uncased-v1")
model = AutoModelForMaskedLM.from_pretrained("dumitrescustefan/bert-base-romanian-uncased-v1")
models_name.append("BERT_Ro")
# set parameters for this model
if_mask = True

In [180]:
honest_score_df, cate_df = test_honest(tokenizer, model, lang,
                                       k_range,if_mask,
                                       honest_score_df, cate_df)

Top_k: 1 language: ro
female honest score:  0.03579952267303103
male honest score:  0.030952380952380953
**********************************************************************
Top_k: 5 language: ro
female honest score:  0.03198090692124105
male honest score:  0.037142857142857144
**********************************************************************
Top_k: 15 language: ro
female honest score:  0.024821002386634844
male honest score:  0.051904761904761905
Percentages of hurtful words about female:  {'ddf': 0.0, 'pa': 0.0, 'asf': 0.051, 'cds': 0.276, 'asm': 0.0, 'qas': 0.019, 'om': 0.0, 'rci': 0.0, 'dmc': 0.006, 'is': 0.0, 'or': 0.045, 'ps': 0.0, 're': 0.0, 'svp': 0.0, 'ddp': 0.006, 'pr': 0.423, 'an': 0.173}
Percentages of hurtful words about male:  {'ddf': 0.0, 'pa': 0.003, 'asf': 0.0, 'cds': 0.419, 'asm': 0.0, 'qas': 0.009, 'om': 0.018, 'rci': 0.003, 'dmc': 0.162, 'is': 0.0, 'or': 0.07, 'ps': 0.0, 're': 0.009, 'svp': 0.028, 'ddp': 0.086, 'pr': 0.107, 'an': 0.086}
**********************

#### RoGPT2

In [ ]:
# import the model and tokenizer 
tokenizer = AutoTokenizer.from_pretrained("readerbench/RoGPT2-medium")
model = AutoModelForCausalLM.from_pretrained("readerbench/RoGPT2-medium")
models_name.append("GPT2_pt")
# set parameters for this model
if_mask = False

In [182]:
honest_score_df, cate_df = test_honest(tokenizer, model, lang,
                                       k_range,if_mask,
                                       honest_score_df, cate_df)

Top_k: 1 language: ro
female honest score:  0.007159904534606206
male honest score:  0.014285714285714285
**********************************************************************
Top_k: 5 language: ro
female honest score:  0.010501193317422435
male honest score:  0.017142857142857144
**********************************************************************
Top_k: 15 language: ro
female honest score:  0.0076372315035799524
male honest score:  0.014603174603174604
Percentages of hurtful words about female:  {'ddf': 0.0, 'pa': 0.0, 'asf': 0.042, 'cds': 0.292, 'asm': 0.0, 'qas': 0.021, 'om': 0.0, 'rci': 0.0, 'dmc': 0.0, 'is': 0.0, 'or': 0.104, 'ps': 0.0, 're': 0.104, 'svp': 0.0, 'ddp': 0.354, 'pr': 0.021, 'an': 0.062}
Percentages of hurtful words about male:  {'ddf': 0.022, 'pa': 0.0, 'asf': 0.0, 'cds': 0.25, 'asm': 0.0, 'qas': 0.043, 'om': 0.0, 'rci': 0.0, 'dmc': 0.065, 'is': 0.0, 'or': 0.033, 'ps': 0.011, 're': 0.0, 'svp': 0.065, 'ddp': 0.446, 'pr': 0.0, 'an': 0.065}
*************************

### Summary_RO

In [183]:
models_name

['BERT_Ro', 'GPT2_pt']

In [184]:
honest_score_df.index = models_name
honest_score_df

,1_female,1_male,5_female,5_male,15_female,15_male
BERT_Ro,0.03580,0.030952,0.031981,0.037143,0.024821,0.051905
GPT2_pt,0.00716,0.014286,0.010501,0.017143,0.007637,0.014603


In [185]:
cate_df.index = [model_name + "_" + g for model_name in models_name for g in ["female","male"]]
cate_df

,ddf,pa,asf,cds,asm,qas,om,rci,dmc,is,or,ps,re,svp,ddp,pr,an
BERT_Ro_female,0.000,0.000,0.051,0.276,0.0,0.019,0.000,0.000,0.006,0.0,0.045,0.000,0.000,0.000,0.006,0.423,0.173
BERT_Ro_male,0.000,0.003,0.000,0.419,0.0,0.009,0.018,0.003,0.162,0.0,0.070,0.000,0.009,0.028,0.086,0.107,0.086
GPT2_pt_female,0.000,0.000,0.042,0.292,0.0,0.021,0.000,0.000,0.000,0.0,0.104,0.000,0.104,0.000,0.354,0.021,0.062
GPT2_pt_male,0.022,0.000,0.000,0.250,0.0,0.043,0.000,0.000,0.065,0.0,0.033,0.011,0.000,0.065,0.446,0.000,0.065


In [186]:
# save test results
save_data(lang, honest_score_df, cate_df)

Finished!
